# Predicting Conversations Gone Awry With Convokit

This interactive tutorial demonstrates how to predict whether a conversation will eventually lead to a personal attack, as seen in the paper [Conversations Gone Awry: Detecting Early Signs of Conversational Failure](http://www.cs.cornell.edu/~cristian/Conversations_gone_awry.html) using the tools provided by convokit. It also serves

In [1]:
import os
import pkg_resources
import json
import itertools
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.feature_selection import f_classif, SelectPercentile
from scipy import stats
from collections import defaultdict
from functools import partial
from multiprocessing import Pool
from itertools import combinations

from convokit import Corpus, QuestionTypology, download, MotifsExtractor, QuestionTypologyUtils, PolitenessStrategies


## Step 1: Load the corpus

In [2]:
# Download the Conversations Gone Awry corpus, provided as part of convokit
if not os.path.exists(os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "conversations-gone-awry-corpus")):
    download("conversations-gone-awry-corpus")

In [3]:
# construct a convokit Corpus object from the downloaded data. The Corpus class provides functionality for
# convenient manipulation of text corpora.
awry_corpus = Corpus(filename=os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "conversations-gone-awry-corpus"))

## Step 2: Extract prompt types features

In this step, we will extract the first of the two types of pragmatic features seen in the paper: prompt types. We can learn prompt types and compute types for each utterance in the corpus using convokit's QuestionTypology class.

In [4]:
# we will train the QuestionTypology object on convokit's wiki corpus. Let's first download the corpus...
if not os.path.exists(os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "wiki-corpus")):
    download("wiki-corpus")

We now train a QuestionTypology object on the downloaded wiki corpus

In [5]:
# some parameters to get us started. Note that "dataset name" refers not to the name of the *source* dataset,
# but the name we want to give to trained output files of the QuestionTypology object. We could have given
# any name we liked - we chose "wiki-corpus-awry" to keep things clear.
dataset_name = "wiki-corpus-awry"
num_clusters = 6

# load the wiki corpus into a convokit Corpus object
data_dir = os.path.join(pkg_resources.resource_filename("convokit", ""), 'downloads')
corpus = Corpus(filename=os.path.join(data_dir, 'wiki-corpus'))

# if this is our first time running this example, we need to fit motifs on the
# dataset. If we have run it before, just load the previously computed motifs
motifs_dir = os.path.join(data_dir, dataset_name + "-motifs")
if not os.path.exists(motifs_dir):
    motifs_dir = None

questionTypology = QuestionTypology(corpus, data_dir, dataset_name=dataset_name, motifs_dir=motifs_dir,
                                    num_dims=50, num_clusters=6, question_threshold=100, answer_threshold=100, 
                                    verbose=1000000, random_seed=2018, questions_only=False, enforce_formatting=False)

building q-a matrices
matrix dir /home/jonathan/research/Cornell-Conversational-Analysis-Toolkit/convokit/downloads/wiki-corpus-awry-matrix exists!
	reading arcs and motifs
	1000000
	2000000
	3000000
	4000000
	5000000
	6000000
	building matrices
	writing stuff
reading question tidxes
reading question leaves
reading answer tidxes
reading question didxes
reading answer didxes
reading question terms
reading answer terms
reading docs
done!


Now that the QuestionTypology object has been trained, we can use it to compute prompt types for our awry corpus (notice that this is a different corpus from what the QuestionTypology object was trained on!). For our purposes, we want the raw features, which are distances from the centers of the KMeans clusters corresponding to each prompt type. We can get these using the `get_qtype_dists` method. Note that in most other situations, where we want just the prompt type, we can use the QuestionTypology object as a Callable, which will return an integer prompt type for each utterance in the corpus. We could also use the `compute_type` function, which is aliased to do the same thing.

In [6]:
prompt_types = questionTypology.get_qtype_dists(awry_corpus)

In [7]:
# let's take a look at what the output looks like
prompt_types.head(10)

,km_0_dist,km_1_dist,km_2_dist,km_3_dist,km_4_dist,km_5_dist,content
146743638.12667.12652,0.940986,0.979943,0.877994,0.896294,0.941438,0.920327,I notice that earier that moved wiki_link to ...
146842219.12874.12874,0.941246,1.128140,1.029870,0.962165,1.082793,0.967436,"Chen was known in the poker world as ""William""..."
146860774.13072.13072,0.962016,1.056367,0.907733,0.911899,1.110880,1.012163,I see what you saying I just read his pokersta...
143890867.11944.11926,1.060801,1.124947,1.148784,0.937019,0.975150,0.942102,No more than two editors advocated deletion. ...
143902946.11991.11991,0.962161,0.853946,0.955107,1.019503,0.766240,0.976699,In the future please don't close Afds when you...
143945536.12065.12065,0.958134,1.042828,1.035856,0.942605,1.055383,0.965266,That simply isn't true. If you read the comme...
144052463.12169.12169,0.946467,1.066047,0.869980,0.832726,1.119957,1.008414,"Somehow, I suspect you may wish to participate..."
144065917.12226.12226,0.969626,0.884449,0.964716,1.009570,0.653972,0.943597,"I assume your deliberate lying has a point, bu..."
127296808.516.516,1.092351,1.014784,1.203938,1.131129,0.693798,1.007752,== Could you stop reverting my corrections ==\n
127296808.534.516,0.945306,0.722245,0.857123,1.035595,0.798275,0.988061,If you have problems with my edits to the 4WD ...


We're going to do a little bit of cleaning up of the prompt types table. First off, we don't need the content column, as we are just interested in using the prompt type features themselves. Second, in the paper we assigned a max distance cutoff, such that distances were capped at 1.

In [8]:
prompt_types = prompt_types.drop(columns="content")
prompt_types[prompt_types > 1] = 1

In [9]:
prompt_types.head(10)

,km_0_dist,km_1_dist,km_2_dist,km_3_dist,km_4_dist,km_5_dist
146743638.12667.12652,0.940986,0.979943,0.877994,0.896294,0.941438,0.920327
146842219.12874.12874,0.941246,1.000000,1.000000,0.962165,1.000000,0.967436
146860774.13072.13072,0.962016,1.000000,0.907733,0.911899,1.000000,1.000000
143890867.11944.11926,1.000000,1.000000,1.000000,0.937019,0.975150,0.942102
143902946.11991.11991,0.962161,0.853946,0.955107,1.000000,0.766240,0.976699
143945536.12065.12065,0.958134,1.000000,1.000000,0.942605,1.000000,0.965266
144052463.12169.12169,0.946467,1.000000,0.869980,0.832726,1.000000,1.000000
144065917.12226.12226,0.969626,0.884449,0.964716,1.000000,0.653972,0.943597
127296808.516.516,1.000000,1.000000,1.000000,1.000000,0.693798,1.000000
127296808.534.516,0.945306,0.722245,0.857123,1.000000,0.798275,0.988061


## Step 3: Extract politeness strategies features

Now we will extract the second type of pragmatic features described in the paper: politeness strategies. We can do this using convokit's PolitenessStrategies class. This class does not require any training, so we can just apply it directly to the corpus.

In [10]:
ps = PolitenessStrategies(awry_corpus)

The politeness strategy features themselves can be accessed via the `feature_df` field of the PolitenessStrategies object.

In [11]:
politeness_strategies = ps.feature_df
politeness_strategies.head(10)

,feature_politeness_==HASHEDGE==,feature_politeness_==HASNEGATIVE==,feature_politeness_==Direct_question==,feature_politeness_==INDICATIVE==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==1st_person==,feature_politeness_==Please_start==,feature_politeness_==Direct_start==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,...,feature_politeness_==Hedges==,feature_politeness_==Deference==,feature_politeness_==1st_person_start==,feature_politeness_==Please==,feature_politeness_==2nd_person==,feature_politeness_==Indirect_(btw)==,feature_politeness_==Factuality==,feature_politeness_==Gratitude==,feature_politeness_==2nd_person_start==,feature_politeness_==HASPOSITIVE==
146743638.12652.12652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146743638.12667.12652,1,1,1,0,0,1,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
146842219.12874.12874,1,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
146860774.13072.13072,1,1,0,0,0,1,0,0,0,0,...,1,0,1,0,1,0,0,1,0,1
143890867.11926.11926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143890867.11944.11926,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
143902946.11991.11991,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
143945536.12065.12065,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
144052463.12169.12169,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
144065917.12226.12226,1,1,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,0


## Step 4: Create pair data

The prediction task defined in the paper is a paired task. The corpus downloaded from convokit already includes metadata about how conversations were paired for the paper, so we don't need to do any of the hard work here. Instead, we'll format the pair information into a table for use in prediction.

In [12]:
# first, we need to directly map conversation IDs to their comments. We'll build a DataFrame to do this
comment_ids = []
convo_ids = []
timestamps = []
page_ids = []
for comment_id in awry_corpus.utterances:
    comment = awry_corpus.utterances[comment_id]
    # section headers are included in the dataset for completeness, but for prediction we need to ignore
    # them as they are not utterances
    if not comment.other["is_section_header"]:
        comment_ids.append(comment_id)
        convo_ids.append(comment.root)
        timestamps.append(comment.timestamp)
        page_ids.append(comment.other["awry_info"]["page_id"])
comment_df = pd.DataFrame({"conversation_id": convo_ids, "timestamp": timestamps, "page_id": page_ids}, index=comment_ids)

In [13]:
# we'll do our construction using awry conversation ID's as the reference key
awry_convo_ids = set()
# these dicts will then all be keyed by awry ID
good_convo_map = {}
page_id_map = {}
for comment in awry_corpus.utterances.values():
    if comment.other["awry_info"]["conversation_has_personal_attack"] and comment.root not in awry_convo_ids:
        awry_convo_ids.add(comment.root)
        good_convo_map[comment.root] = comment.other["awry_info"]["pair_id"]
        page_id_map[comment.root] = comment.other["awry_info"]["page_id"]
awry_convo_ids = list(awry_convo_ids)
pairs_df = pd.DataFrame({"bad_conversation_id": awry_convo_ids,
                         "conversation_id": [good_convo_map[cid] for cid in awry_convo_ids],
                         "page_id": [page_id_map[cid] for cid in awry_convo_ids]})

## Step 5: Construct feature matrix

Now that we have the pair data, we can construct a table of pragmatic features for each pair, to use in prediction. This table will consist of the prompt types and politeness strategies for the first and second comment of each conversation.

In [14]:
def features_for_convo(convo_id):
    
    # first, get the first two comments (IDs) by timestamp in the conversation
    comments_sorted = comment_df[comment_df.conversation_id==convo_id].sort_values(by="timestamp")
    first_id = comments_sorted.iloc[0].name
    second_id = comments_sorted.iloc[1].name
    # get prompt type features
    try:
        first_prompts = prompt_types.loc[first_id]
    except:
        first_prompts = pd.Series(data=np.ones(len(prompt_types.columns)), index=prompt_types.columns)
    try:
        second_prompts = prompt_types.loc[second_id].rename({c: c + "_second" for c in prompt_types.columns})
    except:
        second_prompts = pd.Series(data=np.ones(len(prompt_types.columns)), index=[c + "_second" for c in prompt_types.columns])
    prompts = first_prompts.append(second_prompts)
    # get politeness strategies features
    first_politeness = politeness_strategies.loc[first_id]
    second_politeness = politeness_strategies.loc[second_id].rename({c: c + "_second" for c in politeness_strategies.columns})
    politeness = first_politeness.append(second_politeness)
    return politeness.append(prompts)

In [15]:
convo_ids = pairs_df.bad_conversation_id.append(pairs_df.conversation_id, ignore_index=True)
feats = [features_for_convo(cid) for cid in convo_ids]
feature_table = pd.DataFrame(data=np.vstack([f.values for f in feats]), columns=feats[0].index, index=convo_ids)

In [16]:
# in the paper, we dropped the sentiment lexicon based features (HASPOSITIVE and HASNEGATIVE), opting
# to instead use them as a baseline. We do this here as well to be consistent with the paper.
feature_table = feature_table.drop(columns=["feature_politeness_==HASPOSITIVE==",
                                            "feature_politeness_==HASNEGATIVE==",
                                            "feature_politeness_==HASPOSITIVE==_second",
                                            "feature_politeness_==HASNEGATIVE==_second"])

In [17]:
# let's see how it looks
feature_table.head(5)

,feature_politeness_==HASHEDGE==,feature_politeness_==Direct_question==,feature_politeness_==INDICATIVE==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==1st_person==,feature_politeness_==Please_start==,feature_politeness_==Direct_start==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,feature_politeness_==Indirect_(greeting)==,...,km_2_dist,km_3_dist,km_4_dist,km_5_dist,km_0_dist_second,km_1_dist_second,km_2_dist_second,km_3_dist_second,km_4_dist_second,km_5_dist_second
111356473.3090.3090,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.729520,0.941425,1.000000,1.000000,0.899890,0.839871,0.719525,0.957258,0.906911,0.954653
49083424.9901.9901,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.941099,1.000000,1.000000,0.894193,0.890853,0.799802,0.935380,0.958619,1.000000
219719525.2591.2591,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.866862,0.884670,0.808776,0.873826,1.000000,1.000000,1.000000,1.000000,0.886645,0.994835
180191077.3470.3470,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.869070,0.922005,0.966008,0.902196,0.977763,0.938204,0.899313,0.990845,0.914077,1.000000
418301485.10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.944682,1.000000,0.827365,0.950664,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Step 6: Prediction Utils

We're almost ready to do the prediction! First we need to define a few helper functions...

In [18]:
def run_pred_single(inputs, X, y):
    f_idx, (train_idx, test_idx) = inputs
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    base_clf = Pipeline([("scaler", StandardScaler()), ("featselect", SelectPercentile(f_classif, 10)), ("logreg", LogisticRegression())])
    clf = GridSearchCV(base_clf, {"logreg__C": [10**i for i in range(-4,4)], "featselect__percentile": list(range(10, 110, 10))})

    clf.fit(X_train, y_train)
    
    y_scores = clf.predict_proba(X_test)[:,1]
    y_pred = clf.predict(X_test)
    
    feature_weights = clf.best_estimator_.named_steps["logreg"].coef_.flatten()
    feature_mask = clf.best_estimator_.named_steps["featselect"].get_support()
    
    hyperparams = clf.best_params_
    
    return (y_pred, y_scores, feature_weights, hyperparams, feature_mask)

def run_pred(X, y, fnames, groups):
    feature_weights = {}
    scores = np.asarray([np.nan for i in range(len(y))])
    y_pred = np.zeros(len(y))
    hyperparameters = defaultdict(list)
    splits = list(enumerate(LeaveOneGroupOut().split(X, y, groups)))
    accs = []
        
    with Pool(os.cpu_count()) as p:
        prediction_results = p.map(partial(run_pred_single, X=X, y=y), splits)
        
    fselect_pvals_all = []
    for i in range(len(splits)):
        f_idx, (train_idx, test_idx) = splits[i]
        y_pred_i, y_scores_i, weights_i, hyperparams_i, mask_i = prediction_results[i]
        y_pred[test_idx] = y_pred_i
        scores[test_idx] = y_scores_i
        feature_weights[f_idx] = np.asarray([np.nan for _ in range(len(fnames))])
        feature_weights[f_idx][mask_i] = weights_i
        for param in hyperparams_i:
            hyperparameters[param].append(hyperparams_i[param])   
    
    acc = np.mean(y_pred == y)
    pvalue = stats.binom_test(sum(y_pred == y), n=len(y), alternative="greater")
                
    coef_df = pd.DataFrame(feature_weights, index=fnames)
    coef_df['mean_coef'] = coef_df.apply(np.nanmean, axis=1)
    coef_df['std_coef'] = coef_df.apply(np.nanstd, axis=1)
    return acc, coef_df[['mean_coef', 'std_coef']], scores, pd.DataFrame(hyperparameters), pvalue

def get_labeled_pairs(pairs_df):
    paired_labels = []
    c0s = []
    c1s = []
    page_ids = []
    for i, row in enumerate(pairs_df.itertuples()):
        if i % 2 == 0:
            c0s.append(row.conversation_id)
            c1s.append(row.bad_conversation_id)
        else:
            c0s.append(row.bad_conversation_id)
            c1s.append(row.conversation_id)
        paired_labels.append(i%2)
        page_ids.append(row.page_id)
    return pd.DataFrame({"c0": c0s, "c1": c1s,"first_convo_toxic": paired_labels, "page_id": page_ids})


def mode(seq):
    vals, counts = np.unique(seq, return_counts=True)
    return vals[np.argmax(counts)]

## Step 7: Prediction

And at long last, the prediction task

In [19]:
print("Generating labels...")
labeled_pairs_df = get_labeled_pairs(pairs_df)
print("Retrieving paired features...")
X_c0 = feature_table.loc[labeled_pairs_df.c0].values
X_c1 = feature_table.loc[labeled_pairs_df.c1].values
X = X_c1 - X_c0
y = labeled_pairs_df.first_convo_toxic.values
print("Running leave-one-page-out prediction...")
accuracy, coefs, scores, hyperparams, pvalue = run_pred(X, y, feature_table.columns, labeled_pairs_df.page_id)
print("Accuracy:", accuracy)
print("p-value: %.4e" % pvalue)
print("C (mode):", mode(hyperparams.logreg__C))
print("Percent of features (mode):", mode(hyperparams.featselect__percentile))
print("Coefficents:")
print(coefs.sort_values(by="mean_coef"))

Generating labels...
Retrieving paired features...
Running leave-one-page-out prediction...
Accuracy: 0.6141732283464567
p-value: 4.7715e-09
C (mode): 0.01
Percent of features (mode): 50
Coefficents:
                                                   mean_coef  std_coef
feature_politeness_==2nd_person==_second           -0.170420  0.010455
feature_politeness_==2nd_person_start==_second     -0.138383  0.008210
feature_politeness_==Direct_question==_second      -0.111844  0.006030
feature_politeness_==2nd_person_start==            -0.103344  0.006964
feature_politeness_==Direct_question==             -0.100131  0.005493
feature_politeness_==Indirect_(btw)==              -0.086440  0.003664
feature_politeness_==Please_start==_second         -0.077534  0.004578
km_3_dist_second                                   -0.046964  0.002626
km_0_dist                                          -0.043200  0.006253
feature_politeness_==Factuality==                  -0.033598  0.002096
feature_politeness_

/home/jonathan/miniconda3/envs/convokit/lib/python3.6/site-packages/pandas/core/apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)
/home/jonathan/miniconda3/envs/convokit/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
